In [6]:
import requests
from bs4 import BeautifulSoup
from requests.compat import urljoin, quote_plus, urlparse, unquote
import pandas as pd

In [7]:
quant_page = 1947 // 36 + 1

titles = []
prices = []
authors = []

for i in range(1, quant_page + 1):
  url=f'https://eksmo.ru/khudozhestvennaya-literatura/fantastika/page{i}/?q=&available[]=2&format[]=p'
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  books_sections = soup.find_all('a', class_='book__link')
  for section in books_sections:
    title = section.find('div', class_='book__name').text.strip()
    titles.append(title)

    if section.find('span', class_='book__price-cur'):
      price_block = section.find('span', class_='book__price-cur')
      price = price_block.text.strip()
      only_numbers_price = [x for x in price if x.isdigit()]
      price = int(''.join(only_numbers_price))
    else:
      price = 0
    prices.append(price)

    if section.find('div', class_='book__author'):
      author = section.find('div', class_='book__author').text.strip()
    else:
      author = 'unknown'
    authors.append(author)

In [8]:
df_eksmo = pd.DataFrame({'Название' : titles, 'Автор' : authors, 'Цена' : prices})
df_eksmo

,Название,Автор,Цена
0,Тайна мертвого ректора. Книга 2,Виктор Дашкевич,645
1,Морана и Тень. Плетущая,Лия Арден,680
2,Тайна мертвого ректора. Книга 1,Виктор Дашкевич,645
3,Семь предсказаний Вероники,Аннабель Ли,568
4,Сад эдельвейсов,Лили Мокашь,587
...,...,...,...
1900,Иржина. Случайное – не случайно (с автографом),unknown,440
1901,Уолтон Л. Светлая печаль Авы Лавендер,unknown,512
1902,За тридевять земель (с автографом),unknown,575
1903,Иржина. Случайное – не случайно (с автографом),unknown,440


In [10]:
df_eksmo[df_eksmo['Цена'] == 0]

,Название,Автор,Цена
468,Страна Качества. Qualityland,Марк-Уве Клинг,0
607,Птица малая,Мэри Дориа Расселл,0


У нас всего 2 книги цены, на которые неизвестны (посмотрев на сайте, я понял, что эти книги продаются только тиражами и о цене надо договариваться с издательсвом), поэтому можем пренебречь ими и удалить данные о них соответственно.

In [11]:
df_eksmo = df_eksmo[df_eksmo['Цена'] != 0]
df_eksmo

,Название,Автор,Цена
0,Тайна мертвого ректора. Книга 2,Виктор Дашкевич,645
1,Морана и Тень. Плетущая,Лия Арден,680
2,Тайна мертвого ректора. Книга 1,Виктор Дашкевич,645
3,Семь предсказаний Вероники,Аннабель Ли,568
4,Сад эдельвейсов,Лили Мокашь,587
...,...,...,...
1900,Иржина. Случайное – не случайно (с автографом),unknown,440
1901,Уолтон Л. Светлая печаль Авы Лавендер,unknown,512
1902,За тридевять земель (с автографом),unknown,575
1903,Иржина. Случайное – не случайно (с автографом),unknown,440


In [14]:
df_eksmo.to_csv('eksmo_books_data.csv', index=False, encoding='utf-8')

Посмотрим, какие есть популярные авторы (unknown уберем)

In [13]:
author_counts = df_eksmo.groupby('Автор').size().reset_index(name='Количество книг')
author_counts = author_counts.sort_values(by='Количество книг', ascending=False)
author_counts = author_counts[author_counts['Автор'] != 'unknown']
author_counts.head(20)

,Автор,Количество книг
650,Терри Пратчетт,39
406,Лия Арден,31
479,Мосян Тунсю,29
137,Вадим Панов,24
585,Рэй Брэдбери,23
671,Филип К. Дик,22
173,Галина Гончарова,20
412,Лю Цысинь,19
577,Роджер Желязны,19
273,Евгений Щепетнов,18
